In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

!apt-get update -qq
!apt-get install fonts-nanum* -qq

!pip install kaggle
!pip install -q ultralytics pycocotools easyocr opencv-python pillow matplotlib tqdm scikit-image
!pip install rembg
!pip install onnxruntime onnxruntime-gpu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (4,665 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 125079 files and dire

In [3]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore')

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' # 나눔 고딕
font_name = fm.FontProperties(fname=path, size=10).get_name() # 기본 폰트 사이즈 : 10
plt.rc('font', family=font_name)

fm.fontManager.addfont(path)

In [2]:
# -------------------------------------------------
# 1️⃣ 구글 드라이브 마운트
# -------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# =========================================================
# 셀 0. 공통 설정
# =========================================================
import os, json, random, math, gc, shutil, time
from pathlib import Path
from collections import defaultdict, Counter
import numpy as np
import pandas as pd

from PIL import Image
from tqdm import tqdm

import torch
from ultralytics import YOLO

from sklearn.model_selection import StratifiedKFold
# StratifiedKFold는 scikit-learn 필요
# pip install ultralytics scikit-learn tqdm pillow opencv-python-headless

DATA_ROOT   = "/content/drive/MyDrive/pill_det/data"  # 현재 구조상 ./
IMG_DIR     = os.path.join(DATA_ROOT, "images")        # 중복 제거 끝난 학습 후보 이미지
TEST_DIR    = os.path.join(DATA_ROOT, "test_images")   # 제출용 테스트 이미지
ANN_JSON    = os.path.join(DATA_ROOT, "merged_annotations.json")
WORK_DIR    = "./work_training"
FOLDS       = 3                # Stratified K-Fold 수
EPOCHS      = 150              # 파이널 에폭 (의도적으로 과적합 유도)
BATCH_SIZE  = 32                # GPU 보고 조절
IMGSZ       = 640              # 학습 해상도
MODEL_LIST  = ["yolov8s.pt", "yolov8m.pt"]  # 가벼운 모델 우선. 필요 시 yolov8l.pt 추가

os.makedirs(WORK_DIR, exist_ok=True)

device = 0 if torch.cuda.is_available() else 'cpu'
print("device =>", device)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
device => 0


In [5]:
# =========================================================
# 셀 1. 어노테이션 로드 & 이미지 메타 구성
#   - merged_annotations.json 은 COCO 스타일이라고 가정:
#     { "images":[{id,file_name,width,height}, ...],
#       "annotations":[{image_id,category_id,bbox:[x,y,w,h],...}, ...],
#       "categories":[{id,name,...}, ...]
#     }
# =========================================================
with open(ANN_JSON, "r") as f:
    coco = json.load(f)

images_info = { im["id"]: im for im in coco["images"] }
cats_info   = { c["id"]: c for c in coco["categories"] }

# 이미지별 어노 리스트
img_to_anns = defaultdict(list)
for ann in coco["annotations"]:
    img_to_anns[ann["image_id"]].append({
        "category_id": ann["category_id"],
        "bbox": ann["bbox"]  # [x,y,w,h] in absolute pixel
    })

# 학습 후보 이미지(중복 제거된 images 폴더에 실제 존재하는 것만)
usable_items = []
for img_id, meta in images_info.items():
    fname = meta["file_name"]
    fpath = os.path.join(IMG_DIR, fname)
    if os.path.isfile(fpath):
        anns = img_to_anns.get(img_id, [])
        if len(anns) == 0:
            # 라벨 없는 이미지는 탐지기 학습에 도움 안 되니까 스킵
            continue
        usable_items.append({
            "img_id": img_id,
            "file_name": fname,
            "path": fpath,
            "width":  meta["width"],
            "height": meta["height"],
            "anns":   anns
        })

print("usable images:", len(usable_items))

# Stratified 라벨용 "대표 클래스" 만들기
#  - 그 이미지 안에서 가장 많이 등장한 category_id를 대표로
rep_labels = []
for item in usable_items:
    cls_counts = Counter([a["category_id"] for a in item["anns"]])
    main_cat = cls_counts.most_common(1)[0][0]  # top-1
    rep_labels.append(main_cat)

# numpy로 변환해두면 KFold에 쓰기 편함
img_indices = np.arange(len(usable_items))
rep_labels  = np.array(rep_labels)
print("class dist (대표 라벨):", Counter(rep_labels))

usable images: 644
class dist (대표 라벨): Counter({np.int64(3482): 257, np.int64(3350): 153, np.int64(1899): 97, np.int64(2482): 88, np.int64(3543): 49})


In [6]:
# =========================================================
# 셀 2. Stratified K-Fold split 생성 & YOLO 포맷 세이브 (절대 경로 버전)
# =========================================================
import os, shutil
from pathlib import Path

FOLD_DIRS = []

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

# category_id -> YOLO class index 매핑 (고정)
cat_ids_sorted = sorted(list(cats_info.keys()))  # 예: [1,2,3,...] 같은 COCO cat id들
cid2idx = {cid: idx for idx, cid in enumerate(cat_ids_sorted)}

for fold_idx, (train_idx, val_idx) in enumerate(skf.split(img_indices, rep_labels)):
    fold_root = os.path.join(WORK_DIR, f"fold_{fold_idx}")
    img_train_dir = os.path.join(fold_root, "images/train")
    img_val_dir   = os.path.join(fold_root, "images/val")
    lbl_train_dir = os.path.join(fold_root, "labels/train")
    lbl_val_dir   = os.path.join(fold_root, "labels/val")

    os.makedirs(img_train_dir, exist_ok=True)
    os.makedirs(img_val_dir,   exist_ok=True)
    os.makedirs(lbl_train_dir, exist_ok=True)
    os.makedirs(lbl_val_dir,   exist_ok=True)

    def dump_split(idxs, img_out_dir, lbl_out_dir):
        for i in tqdm(idxs, desc=f"fold{fold_idx} dump_split"):
            item = usable_items[i]
            src_img = item["path"]
            dst_img = os.path.join(img_out_dir, item["file_name"])
            shutil.copy(src_img, dst_img)

            # YOLO label txt 작성
            W = item["width"]; H = item["height"]
            yolo_lines = []
            for a in item["anns"]:
                cid = a["category_id"]           # original COCO category id
                cls_idx = cid2idx[cid]           # 0..nc-1로 매핑
                x,y,w,h = a["bbox"]              # absolute pixel bbox
                cx = (x + w/2)/W
                cy = (y + h/2)/H
                nw = w/W
                nh = h/H
                yolo_lines.append(f"{cls_idx} {cx} {cy} {nw} {nh}")

            dst_lbl = os.path.join(
                lbl_out_dir,
                Path(item["file_name"]).stem + ".txt"
            )
            with open(dst_lbl, "w") as f:
                f.write("\n".join(yolo_lines))

    # 실제 파일 복사 + 라벨 저장
    dump_split(train_idx, img_train_dir, lbl_train_dir)
    dump_split(val_idx,   img_val_dir,   lbl_val_dir)

    # 절대 경로 준비 (Ultralytics가 상대경로 꼬이는 것 방지)
    img_train_dir_abs = os.path.abspath(img_train_dir)
    img_val_dir_abs   = os.path.abspath(img_val_dir)

    # data.yaml 생성
    data_yaml_path = os.path.join(fold_root, "data.yaml")
    with open(data_yaml_path, "w") as f:
        f.write(
            "train: " + img_train_dir_abs + "\n" +
            "val: "   + img_val_dir_abs   + "\n" +
            f"nc: {len(cat_ids_sorted)}\n" +
            "names: " + json.dumps(cat_ids_sorted) + "\n"
        )
        # names: [1,2,3,...] 처럼 cat_ids_sorted를 그대로 넣는다.
        # (대회가 category_id 그대로 쓰는 구조라면 이게 편하다)

    FOLD_DIRS.append({
        "fold": fold_idx,
        "root": fold_root,
        "data_yaml": data_yaml_path,
        "train_n": len(train_idx),
        "val_n": len(val_idx),
        "cat_ids_sorted": cat_ids_sorted,  # 나중 csv 복원용
    })

print("FOLD_DIRS summary:")
for info in FOLD_DIRS:
    print(info)

fold2 dump_split: 100%|██████████| 214/214 [00:02<00:00, 96.10it/s]

FOLD_DIRS summary:
{'fold': 0, 'root': './work_training/fold_0', 'data_yaml': './work_training/fold_0/data.yaml', 'train_n': 429, 'val_n': 215, 'cat_ids_sorted': [1899, 2482, 3350, 3482, 3543, 3742, 3831, 4377, 4542, 5093, 5885, 6191, 6562, 10220, 12080, 12246, 12419, 12777, 13394, 13899, 16231, 16261, 16547, 16550, 16687, 18109, 18146, 18356, 19231, 19551, 19606, 19860, 20013, 20237, 20876, 21025, 21324, 21770, 22073, 22346, 22361, 22626, 23202, 23222, 24849, 25366, 25437, 25468, 27652, 27732, 27776, 27925, 27992, 28762, 29344, 29450, 29666, 29870, 30307, 31704, 31862, 31884, 32309, 33008, 33207, 33877, 33879, 34596, 35205, 36636, 38161, 41767, 44198]}
{'fold': 1, 'root': './work_training/fold_1', 'data_yaml': './work_training/fold_1/data.yaml', 'train_n': 429, 'val_n': 215, 'cat_ids_sorted': [1899, 2482, 3350, 3482, 3543, 3742, 3831, 4377, 4542, 5093, 5885, 6191, 6562, 10220, 12080, 12246, 12419, 12777, 13394, 13899, 16231, 16261, 16547, 16550, 16687, 18109, 18146, 18356, 19231, 1955

In [7]:
# =========================================================
# 셀 3. Fold 단위 학습 함수 (과적합 세팅 + 결과 CSV→JSON 로깅)
# =========================================================
import os, time, json
from pathlib import Path
import pandas as pd
from ultralytics import YOLO

def train_one_fold(fold_info, base_model="yolov8s.pt"):
    """
    fold_info: FOLD_DIRS[i]에서 온 dict
      {
        "fold": fold_idx,
        "root": fold_root,
        "data_yaml": data_yaml_path,
        "train_n": ...,
        "val_n": ...
      }

    return:
      {
        "fold": fold_idx,
        "model": base_model,
        "run_name": out_runname,
        "run_dir": <ultralytics run dir>,
        "best_pt": ".../best.pt" or None,
        "last_pt": ".../last.pt" or None,
        "history_json": "..._history.json"
      }
    """

    fold_idx    = fold_info["fold"]
    data_yaml   = fold_info["data_yaml"]

    # Ultralytics run name 예: fold0_yolov8s
    out_runname = f"fold{fold_idx}_{Path(base_model).stem}"

    # Ultralytics는 project/name 조합으로 디렉토리를 만든다.
    # project=WORK_DIR, name=out_runname → {WORK_DIR}/{out_runname}/...
    run_dir = os.path.join(WORK_DIR, out_runname)
    os.makedirs(run_dir, exist_ok=True)

    print(f"🚀 START TRAIN fold={fold_idx} model={base_model} -> {out_runname}")

    # 모델 로드
    model = YOLO(base_model)

    # 학습 (콜백 없이)
    # photometric augmentation 강하게, geom 약하게, 과적합 유도용 긴 epochs
    results = model.train(
        data=data_yaml,
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH_SIZE,
        device=device,
        project=WORK_DIR,
        name=out_runname,
        workers=2,
        patience=50,
        verbose=True,
        deterministic=True,

        mosaic=0.0,
        mixup=0.0,
        copy_paste=0.05,   # 소수 클래스 부스팅
        fliplr=0.5,
        flipud=0.1,
        degrees=5.0,
        translate=0.04,
        scale=0.15,
        shear=3.0,
        hsv_h=0.02,
        hsv_s=0.45,
        hsv_v=0.35,
    )

    # ---------------------------------
    # 1) results.csv -> history_json으로 변환
    # ---------------------------------
    # Ultralytics는 run_dir 안 또는 run_dir 상위에 results.csv를 만든다.
    # 일반적으로 {WORK_DIR}/{out_runname}/results.csv
    csv_path_1 = os.path.join(WORK_DIR, out_runname, "results.csv")
    csv_path_2 = os.path.join(run_dir, "results.csv")

    if os.path.isfile(csv_path_1):
        csv_path = csv_path_1
    elif os.path.isfile(csv_path_2):
        csv_path = csv_path_2
    else:
        csv_path = None

    hist_path = os.path.join(run_dir, f"{out_runname}_history.json")
    epoch_logs = []

    if csv_path and os.path.isfile(csv_path):
        df_csv = pd.read_csv(csv_path)
        # df_csv 보통 'epoch','train/box_loss','metrics/mAP50-95(B)' 등의 열이 있다.
        # 버전에 따라 컬럼명이 다를 수 있어서 그냥 전체 row를 dict로 기록한다.
        for _, row in df_csv.iterrows():
            epoch_logs.append({
                "epoch": int(row["epoch"]) if "epoch" in row else None,
                "row": row.to_dict(),
                "time": time.time()
            })
    else:
        # csv가 없으면 fallback으로 results 객체만 저장
        # results 객체는 dict-like이지만 epoch별 세부 로그는 없다
        epoch_logs.append({
            "epoch": None,
            "row": getattr(results, "results_dict", {}),
            "time": time.time()
        })

    with open(hist_path, "w") as f:
        json.dump(epoch_logs, f, indent=2)
    print(f"✅ saved history JSON -> {hist_path}")

    # ---------------------------------
    # 2) best.pt / last.pt 경로 추출
    # ---------------------------------
    best_pt_path = os.path.join(WORK_DIR, out_runname, "weights", "best.pt")
    last_pt_path = os.path.join(WORK_DIR, out_runname, "weights", "last.pt")

    if not os.path.isfile(best_pt_path):
        best_pt_path = None
    if not os.path.isfile(last_pt_path):
        last_pt_path = None

    return {
        "fold": fold_idx,
        "model": base_model,
        "run_name": out_runname,
        "run_dir": os.path.join(WORK_DIR, out_runname),
        "best_pt": best_pt_path,
        "last_pt": last_pt_path,
        "history_json": hist_path,
    }

# =========================================================
# 모든 폴드 × 모든 모델 학습 실행
# =========================================================
train_artifacts = []

for fold in FOLD_DIRS:
    for mname in MODEL_LIST:
        art = train_one_fold(fold, base_model=mname)
        train_artifacts.append(art)

print("📦 train_artifacts (YOLO):")
for a in train_artifacts:
    print(a)

🚀 START TRAIN fold=0 model=yolov8s.pt -> fold0_yolov8s
Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./work_training/fold_0/data.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.45, hsv_v=0.35, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=fold0_yolov8s2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto,

In [8]:
# =========================================================
# 셀 0. rebuild_train_artifacts_from_fs
# =========================================================
import os, glob
from pathlib import Path

def rebuild_train_artifacts_from_fs(work_dir, model_list, folds):
    """
    이전에 학습된 YOLO run들을 스캔해서 artifact 메타데이터를 복원한다.
    ex) ./work_training/fold0_yolov8s2/weights/best.pt
    """
    rebuilt = []
    all_run_dirs = [d for d in glob.glob(os.path.join(work_dir, "*")) if os.path.isdir(d)]

    for run_dir in all_run_dirs:
        run_name = os.path.basename(run_dir)
        if not run_name.startswith("fold"):
            continue

        try:
            after_fold = run_name[len("fold"):]     # e.g. '0_yolov8s2'
            fold_str = after_fold.split("_", 1)[0]  # '0'
            fold_idx = int(fold_str)
        except Exception:
            continue

        # 모델명 매칭
        after_foldnum = after_fold.split("_", 1)[1]
        model_match = None
        for m in model_list:
            stem = Path(m).stem
            if after_foldnum.startswith(stem):
                model_match = m
                break
        if model_match is None:
            continue

        best_pt = os.path.join(run_dir, "weights", "best.pt")
        last_pt = os.path.join(run_dir, "weights", "last.pt")
        hist_json = os.path.join(run_dir, f"{run_name}_history.json")

        # history 파일이 없을 경우 fallback
        if not os.path.isfile(hist_json):
            alt_hist = glob.glob(os.path.join(run_dir, "*_history.json"))
            hist_json = alt_hist[0] if alt_hist else None

        rebuilt.append({
            "fold": fold_idx,
            "model": model_match,
            "run_name": run_name,
            "run_dir": run_dir,
            "best_pt": best_pt if os.path.isfile(best_pt) else None,
            "last_pt": last_pt if os.path.isfile(last_pt) else None,
            "history_json": hist_json if hist_json and os.path.isfile(hist_json) else None
        })

    # 최신 순 정렬
    rebuilt.sort(key=lambda x: os.path.getmtime(x["run_dir"]), reverse=True)
    return rebuilt

# 실행
WORK_DIR = "./work_training"   # 작업 디렉토리 (기존과 동일하게)
MODEL_LIST = ["yolov8s.pt", "yolov8m.pt"]  # 학습에 사용한 모델명 리스트
FOLD_DIRS = [{"fold": i, "root": f"./fold_{i}"} for i in range(3)]  # fold 수에 맞게

rebuilt_artifacts = rebuild_train_artifacts_from_fs(WORK_DIR, MODEL_LIST, folds=range(len(FOLD_DIRS)))

print("🔥 rebuilt_artifacts:")
for r in rebuilt_artifacts:
    print(r)

🔥 rebuilt_artifacts:
{'fold': 2, 'model': 'yolov8m.pt', 'run_name': 'fold2_yolov8m', 'run_dir': './work_training/fold2_yolov8m', 'best_pt': None, 'last_pt': None, 'history_json': './work_training/fold2_yolov8m/fold2_yolov8m_history.json'}
{'fold': 2, 'model': 'yolov8m.pt', 'run_name': 'fold2_yolov8m2', 'run_dir': './work_training/fold2_yolov8m2', 'best_pt': './work_training/fold2_yolov8m2/weights/best.pt', 'last_pt': './work_training/fold2_yolov8m2/weights/last.pt', 'history_json': None}
{'fold': 2, 'model': 'yolov8s.pt', 'run_name': 'fold2_yolov8s', 'run_dir': './work_training/fold2_yolov8s', 'best_pt': None, 'last_pt': None, 'history_json': './work_training/fold2_yolov8s/fold2_yolov8s_history.json'}
{'fold': 2, 'model': 'yolov8s.pt', 'run_name': 'fold2_yolov8s2', 'run_dir': './work_training/fold2_yolov8s2', 'best_pt': './work_training/fold2_yolov8s2/weights/best.pt', 'last_pt': './work_training/fold2_yolov8s2/weights/last.pt', 'history_json': None}
{'fold': 1, 'model': 'yolov8m.pt', 

In [9]:
import os, glob
import pandas as pd

def pick_best_run_by_map(artifacts, metric_col="metrics/mAP50-95(B)"):
    """
    artifacts: rebuilt_artifacts 리스트
    metric_col: results.csv 안에서 우리가 보고 싶은 컬럼명
      - 버전에 따라 'metrics/mAP50-95(B)' 또는 'metrics/mAP50-95(B)@0.5:0.95' 등 변형 가능
      - 안 맞으면 fallback으로 'metrics/mAP50-95(B)' 대신 'metrics/mAP50-95(B)' 비슷한 후보를 찾는다
    return: best_artifact(딕셔너리), best_score(float)
    """

    best_art = None
    best_score = -1.0

    for art in artifacts:
        csv_path = os.path.join(art["run_dir"], "results.csv")
        if not os.path.isfile(csv_path):
            continue
        df = pd.read_csv(csv_path)

        # metric_col이 없을 수도 있으니까 유사 후보를 찾는다
        if metric_col not in df.columns:
            # 후보 스캔
            cand_cols = [c for c in df.columns if "mAP" in c and "50" in c]
            # 예: ['metrics/mAP50(B)', 'metrics/mAP50-95(B)']
            if len(cand_cols) == 0:
                # 이 run은 스킵
                continue
            metric_use = cand_cols[-1]  # 가장 복잡한(mAP50-95) 쪽을 골라본다
        else:
            metric_use = metric_col

        # 마지막 epoch 기준으로 스코어 뽑기
        last_row = df.sort_values("epoch").iloc[-1]
        score = float(last_row[metric_use])

        if score > best_score:
            best_score = score
            best_art = art

    return best_art, best_score

best_artifact, best_score = pick_best_run_by_map(rebuilt_artifacts)
print("🏆 best artifact:", best_artifact)
print("🏆 best score:", best_score)

🏆 best artifact: {'fold': 1, 'model': 'yolov8m.pt', 'run_name': 'fold1_yolov8m2', 'run_dir': './work_training/fold1_yolov8m2', 'best_pt': './work_training/fold1_yolov8m2/weights/best.pt', 'last_pt': './work_training/fold1_yolov8m2/weights/last.pt', 'history_json': None}
🏆 best score: 0.97054


In [16]:

import shutil
from pathlib import Path

def build_fulltrain_from_fold(fold_info, out_root):
    """
    fold_info: FOLD_DIRS[fold_idx] 형태 (fold, root, data_yaml, ...)
    out_root: 새 fulltrain 디렉토리 (예: ./work_training/fulltrain_fold0)
    결과 구조:
      out_root/images/train/*.jpg (train+val 전체)
      out_root/labels/train/*.txt (train+val 전체)
      out_root/data.yaml (train만 가리킴)
    """

    fold_root = os.path.join(WORK_DIR, fold_info["root"])
    train_img_dir = os.path.join(fold_root, "images/train")
    val_img_dir   = os.path.join(fold_root, "images/val")
    train_lbl_dir = os.path.join(fold_root, "labels/train")
    val_lbl_dir   = os.path.join(fold_root, "labels/val")

    img_out_dir = os.path.join(out_root, "images/train")
    lbl_out_dir = os.path.join(out_root, "labels/train")
    os.makedirs(img_out_dir, exist_ok=True)
    os.makedirs(lbl_out_dir, exist_ok=True)

    def copy_all(src_img_dir, src_lbl_dir):
        for fname in os.listdir(src_img_dir):
            if not fname.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp")):
                continue
            # 이미지 복사
            shutil.copy(
                os.path.join(src_img_dir, fname),
                os.path.join(img_out_dir, fname)
            )
            # 라벨 복사
            stem = Path(fname).stem
            lbl_src = os.path.join(src_lbl_dir, stem + ".txt")
            lbl_dst = os.path.join(lbl_out_dir, stem + ".txt")
            if os.path.isfile(lbl_src):
                shutil.copy(lbl_src, lbl_dst)
            else:
                # 라벨 없는 파일은 그냥 스킵해도 됨
                pass

    # train+val를 합쳐서 train으로
    copy_all(train_img_dir, train_lbl_dir)
    copy_all(val_img_dir, val_lbl_dir)

    # cid2idx / cat_ids_sorted는 셀2에서 썼던 로직 그대로
    cat_ids_sorted = sorted(list(cats_info.keys()))

    data_yaml_full = os.path.join(out_root, "data.yaml")
    with open(data_yaml_full, "w") as f:
        f.write(
            f"train: {img_out_dir}\n"
            f"val: {img_out_dir}\n"    # full-train에선 val도 동일하게 자기 자신을 가리켜 과적합을 유도
            f"nc: {len(cat_ids_sorted)}\n"
            f"names: {cat_ids_sorted}\n"
        )

    return data_yaml_full

# best_fold_idx 가져오기
best_fold_idx = best_artifact["fold"]
best_fold_info = None
for fi in FOLD_DIRS:
    if fi["fold"] == best_fold_idx:
        best_fold_info = fi
        break

assert best_fold_info is not None, "best_fold_info 못 찾음"

FULLTRAIN_ROOT = os.path.join(WORK_DIR, f"fulltrain_fold{best_fold_idx}")
os.makedirs(FULLTRAIN_ROOT, exist_ok=True)

fulltrain_yaml = build_fulltrain_from_fold(best_fold_info, FULLTRAIN_ROOT)
print("📂 full-train yaml:", fulltrain_yaml)

📂 full-train yaml: ./work_training/fulltrain_fold1/data.yaml


In [20]:
import os, shutil
from pathlib import Path

def build_fulltrain_from_fold_abs(fold_info, out_root):
    """
    fold_info 예:
      {
        "fold": 1,
        "root": "./work_training/fold_1",   # <= 반드시 이런 구조
        ...
      }

    out_root 예:
      ./work_training/fulltrain_fold1

    결과:
      out_root/images/train/*.jpg   (train+val 전체 합침)
      out_root/labels/train/*.txt
      out_root/data.yaml            (절대경로로 작성)
    """

    fold_root = fold_info["root"]          # ex) "./work_training/fold_1"
    fold_root_abs = os.path.abspath(fold_root)

    train_img_dir = os.path.join(fold_root_abs, "images", "train")
    val_img_dir   = os.path.join(fold_root_abs, "images", "val")
    train_lbl_dir = os.path.join(fold_root_abs, "labels", "train")
    val_lbl_dir   = os.path.join(fold_root_abs, "labels", "val")

    out_root_abs = os.path.abspath(out_root)
    img_out_dir  = os.path.join(out_root_abs, "images", "train")
    lbl_out_dir  = os.path.join(out_root_abs, "labels", "train")
    os.makedirs(img_out_dir, exist_ok=True)
    os.makedirs(lbl_out_dir, exist_ok=True)

    def copy_all(src_img_dir, src_lbl_dir):
        if not os.path.isdir(src_img_dir):
            return
        for fname in os.listdir(src_img_dir):
            if not fname.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp")):
                continue

            src_img_path = os.path.join(src_img_dir, fname)
            dst_img_path = os.path.join(img_out_dir, fname)
            shutil.copy(src_img_path, dst_img_path)

            stem = Path(fname).stem
            src_lbl_path = os.path.join(src_lbl_dir, stem + ".txt")
            dst_lbl_path = os.path.join(lbl_out_dir, stem + ".txt")
            if os.path.isfile(src_lbl_path):
                shutil.copy(src_lbl_path, dst_lbl_path)
            # 라벨 없으면 그냥 패스

    # train+val 전부 train으로 몰아넣음 (full overfit)
    copy_all(train_img_dir, train_lbl_dir)
    copy_all(val_img_dir,   val_lbl_dir)

    # YOLO data.yaml을 절대경로로 쓴다
    img_out_dir_abs = os.path.abspath(img_out_dir)

    cat_ids_sorted = sorted(list(cats_info.keys()))  # cats_info는 셀1 전역

    data_yaml_full = os.path.join(out_root_abs, "data.yaml")
    with open(data_yaml_full, "w") as f:
        f.write(
            f"train: {img_out_dir_abs}\n"
            f"val: {img_out_dir_abs}\n"   # full-train에서는 val도 train과 동일하게(=완전 과적합)
            f"nc: {len(cat_ids_sorted)}\n"
            f"names: {cat_ids_sorted}\n"
        )

    return data_yaml_full

# 다시 만들기
best_fold_idx = best_artifact["fold"]  # ex) 1
best_fold_info = None
for fi in FOLD_DIRS:
    if fi["fold"] == best_fold_idx:
        best_fold_info = fi
        break

if best_fold_info is None:
    raise RuntimeError("best_fold_info 못 찾음. FOLD_DIRS 내용을 확인해봐.")

FULLTRAIN_ROOT = os.path.join(WORK_DIR, f"fulltrain_fold{best_fold_idx}")
os.makedirs(FULLTRAIN_ROOT, exist_ok=True)

fulltrain_yaml_abs = build_fulltrain_from_fold_abs(best_fold_info, FULLTRAIN_ROOT)
print("📂 full-train yaml (ABS):", fulltrain_yaml_abs)

📂 full-train yaml (ABS): /content/work_training/fulltrain_fold1/data.yaml


In [21]:
from ultralytics import YOLO
import pandas as pd
import time, json, os
from pathlib import Path  # ✅ 누락 보완

FINAL_EPOCHS = 300   # 의도적으로 과적합
# best_artifact["model"] 예: "yolov8m.pt"
# Path(...).stem -> "yolov8m"
FINAL_MODEL  = Path(best_artifact["model"]).stem
FINAL_RUNNAME = f"final_overfit_fold{best_fold_idx}_{FINAL_MODEL}"

print(f"🔥 FINAL OVERFIT TRAIN: {FINAL_RUNNAME}")

# =========================================================
# 어떤 weight로 시작할지 결정
# - best_artifact["best_pt"]가 있으면 그걸로 로드해서 이어붙이는 게 가장 좋음
# - 없으면 그냥 base model 이름(e.g. "yolov8m.pt")로 초기화
# =========================================================
start_weight = None
if "best_pt" in best_artifact and best_artifact["best_pt"] and os.path.isfile(best_artifact["best_pt"]):
    start_weight = best_artifact["best_pt"]
elif "last_pt" in best_artifact and best_artifact["last_pt"] and os.path.isfile(best_artifact["last_pt"]):
    start_weight = best_artifact["last_pt"]
else:
    # fallback: pretrained checkpoint ex) "yolov8m.pt"
    start_weight = best_artifact["model"]

print("🚀 using start_weight ->", start_weight)

final_model = YOLO(start_weight)

# =========================================================
# fulltrain_yaml 은 전체 train+val 합친 yaml
# 주의: yaml 내부의 경로는 절대경로(or Colab 현재 경로 기준의 상대경로)로 올바르게 지정돼 있어야 함
# =========================================================
results_final = final_model.train(
    data=fulltrain_yaml,
    epochs=FINAL_EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH_SIZE,
    device=device,
    project=WORK_DIR,
    name=FINAL_RUNNAME,
    workers=2,
    patience=FINAL_EPOCHS,   # 사실상 early stop 안 하게
    verbose=True,
    deterministic=True,

    # 🔥 과적합 유도 세팅 (photometric 강, geom 조금)
    mosaic=0.0,
    mixup=0.0,
    copy_paste=0.10,
    fliplr=0.5,
    flipud=0.2,
    degrees=8.0,
    translate=0.06,
    scale=0.20,
    shear=4.0,
    hsv_h=0.02,
    hsv_s=0.5,
    hsv_v=0.4,
)

# =========================================================
# history 저장
# =========================================================
final_run_dir   = os.path.join(WORK_DIR, FINAL_RUNNAME)
final_csv_path  = os.path.join(final_run_dir, "results.csv")
final_hist_path = os.path.join(final_run_dir, f"{FINAL_RUNNAME}_history.json")

epoch_logs = []
if os.path.isfile(final_csv_path):
    df_csv = pd.read_csv(final_csv_path)
    for _, row in df_csv.iterrows():
        epoch_logs.append({
            "epoch": int(row["epoch"]) if "epoch" in row else None,
            "row": row.to_dict(),
            "time": time.time()
        })

with open(final_hist_path, "w") as f:
    json.dump(epoch_logs, f, indent=2)

print("📝 final history saved:", final_hist_path)

# =========================================================
# 최종 가중치 경로 (OCR 재랭킹·앙상블 파이프라인에서 inference용으로 쓸 것)
# =========================================================
FINAL_BEST_PT = os.path.join(final_run_dir, "weights", "best.pt")
FINAL_LAST_PT = os.path.join(final_run_dir, "weights", "last.pt")

if not os.path.isfile(FINAL_BEST_PT):
    FINAL_BEST_PT = None
if not os.path.isfile(FINAL_LAST_PT):
    FINAL_LAST_PT = None

print("🏁 FINAL_BEST_PT :", FINAL_BEST_PT)
print("🏁 FINAL_LAST_PT :", FINAL_LAST_PT)
print("🏁 FINAL_RUNNAME :", FINAL_RUNNAME)

# 이제 다음 셀에서 이 값(FINAL_BEST_PT or FINAL_LAST_PT 중 존재하는 것)을
# infer_yolo_weight_over_test(...)에 넣어서
# OCR 재랭킹 + Weighted Boxes Fusion + 최종 submission.csv 생성으로 넘어가면 된다.

🔥 FINAL OVERFIT TRAIN: final_overfit_fold1_yolov8m
🚀 using start_weight -> ./work_training/fold1_yolov8m2/weights/best.pt
Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./work_training/fulltrain_fold1/data.yaml, degrees=8.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.5, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=./work_training/fold1_yolov8m2/weights/best.pt, momentum=0.937, mosai

In [24]:
# =========================================================
# 셀 1. final_overfit_fold1_yolov8m3 best.pt 추론 결과 수집
# =========================================================
import os
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO

# 너의 최종 러너 이름과 weight 경로
FINAL_RUNNAME = "final_overfit_fold1_yolov8m3"
FINAL_WEIGHT  = f"./work_training/{FINAL_RUNNAME}/weights/best.pt"

# cats_info에서 카테고리 순서를 YOLO 학습과 동일하게 복원
# (우리가 YOLO 학습할 때 cid2idx = {cid: idx for idx, cid in enumerate(sorted(cats_info.keys()))})
cat_ids_sorted = sorted(list(cats_info.keys()))
idx2cid = {idx: cid for idx, cid in enumerate(cat_ids_sorted)}

def infer_single_model(test_dir, weight_path, img_size, idx2cid_map):
    """
    test_dir의 모든 이미지를 weight_path 모델로 추론해서 dict 반환
    out[img_path] = {
        "boxes": (N,4) float32 [x1,y1,x2,y2],
        "scores": (N,) float32,
        "cat_ids": (N,) int32  # original category_id
    }
    """
    model = YOLO(weight_path)
    out_map = {}

    test_files = sorted([
        f for f in os.listdir(test_dir)
        if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))
    ])

    for fn in tqdm(test_files, desc="[FINAL MODEL INFER]"):
        ip = os.path.join(test_dir, fn)
        pred = model.predict(
            source=ip,
            imgsz=img_size,
            conf=0.001,       # 낮춰서 최대한 많이 뽑기
            iou=0.7,
            agnostic_nms=True,
            verbose=False
        )[0]

        if pred.boxes is None or len(pred.boxes) == 0:
            out_map[ip] = {
                "boxes": np.zeros((0,4), dtype=np.float32),
                "scores": np.zeros((0,), dtype=np.float32),
                "cat_ids": np.zeros((0,), dtype=np.int32),
            }
            continue

        boxes_xyxy = pred.boxes.xyxy.cpu().numpy().astype(np.float32)
        scores     = pred.boxes.conf.cpu().numpy().astype(np.float32)
        classes    = pred.boxes.cls.cpu().numpy().astype(int)

        cat_ids = np.array(
            [idx2cid_map[c] for c in classes],
            dtype=np.int32
        )

        out_map[ip] = {
            "boxes": boxes_xyxy,
            "scores": scores,
            "cat_ids": cat_ids
        }

    return out_map

det_map = infer_single_model(TEST_DIR, FINAL_WEIGHT, IMGSZ, idx2cid)
print("inference done. images:", len(det_map))

[FINAL MODEL INFER]: 100%|██████████| 843/843 [00:58<00:00, 14.46it/s]

inference done. images: 843


In [27]:
# =========================================================
# 셀 2. OCR 기반 점수 보정
# =========================================================
import cv2
import easyocr
import torch
from pathlib import Path
import numpy as np
from tqdm import tqdm

# EasyOCR 리더 준비 (영문/숫자 가정)
reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())

# category_id -> class_name 매핑
cid_to_name = {cid: info["name"] for cid, info in cats_info.items()}

def ocr_scores_for_image(img_path, boxes_xyxy, class_names):
    """
    boxes_xyxy: (N,4) [x1,y1,x2,y2]
    class_names: ['약A','약B',...] 길이 N
    return: np.array shape (N,) OCR 신뢰도 (0~1 근사)
    """
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        return np.zeros((len(boxes_xyxy),), dtype=np.float32)

    H, W = img_bgr.shape[:2]
    scores_ocr = []

    for (x1,y1,x2,y2), cname in zip(boxes_xyxy, class_names):
        xx1 = max(0, int(x1))
        yy1 = max(0, int(y1))
        xx2 = min(W-1, int(x2))
        yy2 = min(H-1, int(y2))

        if xx2 <= xx1 or yy2 <= yy1:
            scores_ocr.append(0.0)
            continue

        crop = img_bgr[yy1:yy2, xx1:xx2]
        if crop.size == 0:
            scores_ocr.append(0.0)
            continue

        h_c, w_c = crop.shape[:2]
        if h_c < 12 or w_c < 12:
            # 너무 작으면 OCR 무의미
            scores_ocr.append(0.0)
            continue

        crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        result = reader.readtext(crop_rgb, detail=1)

        if len(result) == 0:
            scores_ocr.append(0.0)
        else:
            # 가장 높은 conf 사용
            best_conf = max([det[2] for det in result])
            scores_ocr.append(float(best_conf))

    return np.array(scores_ocr, dtype=np.float32)

def fuse_det_and_ocr(det_map, det_w=0.8, ocr_w=0.2, topK=4):
    """
    det_map[ip] = { boxes, scores, cat_ids }
    return rows(list[dict]) for CSV
    """
    rows = []
    annotation_id = 1

    test_files = sorted([
        f for f in os.listdir(TEST_DIR)
        if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))
    ])

    for fn in tqdm(test_files, desc="[OCR+DET to CSV rows]"):
        ip = os.path.join(TEST_DIR, fn)
        pred = det_map.get(ip, None)

        if pred is None or len(pred["boxes"]) == 0:
            # 박스 없으면 그냥 스킵하거나 빈 row를 넣는지 네 대회 규칙에 맞춰야 하는데
            # 지금은 스킵
            continue

        boxes_xyxy = pred["boxes"]
        det_scores = pred["scores"]
        cat_ids    = pred["cat_ids"]

        class_names = [cid_to_name[int(c)] for c in cat_ids]
        ocr_scores  = ocr_scores_for_image(ip, boxes_xyxy, class_names)

        fused_scores = det_w * det_scores + ocr_w * ocr_scores

        # score 높은 순으로 topK개만
        order = np.argsort(-fused_scores)[:topK]

        base_name = Path(fn).stem  # 확장자 제거
        for k in order:
            x1, y1, x2, y2 = boxes_xyxy[k]
            w = x2 - x1
            h = y2 - y1

            # 디버깅 출력(원하면 주석처리 가능)
            print(round(float(det_scores[k]),6), "DET")
            print(round(float(ocr_scores[k]),6), "OCR")
            print(round(float(fused_scores[k]),6), "OCR+DET")

            rows.append({
                "annotation_id": annotation_id,
                "image_id": base_name,
                "category_id": int(cat_ids[k]),  # 원래 category_id
                "bbox_x": round(float(x1),2),
                "bbox_y": round(float(y1),2),
                "bbox_w": round(float(w),2),
                "bbox_h": round(float(h),2),
                "score":  round(float(fused_scores[k]),6),
            })
            annotation_id += 1

    return rows

rows_for_csv = fuse_det_and_ocr(det_map, det_w=0.95, ocr_w=0.05, topK=4)
print("총 rows:", len(rows_for_csv))

[OCR+DET to CSV rows]:   0%|          | 3/843 [00:00<01:18, 10.74it/s]

0.967003 DET
0.3703 OCR
0.937168 OCR+DET
0.97181 DET
0.263445 OCR
0.936392 OCR+DET
0.976711 DET
0.0 OCR
0.927875 OCR+DET
0.961895 DET
0.092287 OCR
0.918414 OCR+DET
0.985396 DET
0.0 OCR
0.936126 OCR+DET
0.971577 DET
0.164116 OCR
0.931204 OCR+DET
0.973846 DET
0.0 OCR
0.925153 OCR+DET
0.965672 DET
0.0 OCR
0.917388 OCR+DET
0.974597 DET
0.716805 OCR
0.961707 OCR+DET
0.973401 DET
0.426072 OCR
0.946034 OCR+DET
0.983167 DET
0.0 OCR
0.934009 OCR+DET
0.978847 DET
0.0 OCR
0.929905 OCR+DET


[OCR+DET to CSV rows]:   1%|          | 5/843 [00:00<01:23, 10.03it/s]

0.971139 DET
0.934183 OCR
0.969292 OCR+DET
0.975724 DET
0.332263 OCR
0.943551 OCR+DET
0.97175 DET
0.178106 OCR
0.932068 OCR+DET
0.940469 DET
0.483495 OCR
0.91762 OCR+DET
0.977378 DET
0.726521 OCR
0.964835 OCR+DET
0.969675 DET
0.545945 OCR
0.948489 OCR+DET
0.984753 DET
0.0 OCR
0.935516 OCR+DET
0.982385 DET
0.0 OCR
0.933266 OCR+DET


[OCR+DET to CSV rows]:   1%|          | 7/843 [00:00<01:28,  9.45it/s]

...

0.970693 DET
0.344103 OCR
0.939363 OCR+DET
0.980064 DET
0.0 OCR
0.931061 OCR+DET
0.975197 DET
0.0 OCR
0.926437 OCR+DET
0.973908 DET
0.0 OCR
0.925213 OCR+DET
0.97419 DET
0.223327 OCR
0.936646 OCR+DET
0.978193 DET
0.0 OCR
0.929284 OCR+DET
0.959862 DET
0.0 OCR
0.911869 OCR+DET
0.953179 DET
0.0 OCR
0.90552 OCR+DET
0.98305 DET
0.0 OCR
0.933898 OCR+DET
0.974276 DET
0.122025 OCR
0.931664 OCR+DET
0.976305 DET
0.0 OCR
0.92749 OCR+DET
0.976064 DET
0.0 OCR
0.927261 OCR+DET


[OCR+DET to CSV rows]: 100%|█████████▉| 839/843 [01:21<00:00, 11.20it/s]

0.970712 DET
0.983446 OCR
0.971349 OCR+DET
0.973552 DET
0.402837 OCR
0.945016 OCR+DET
0.983446 DET
0.0 OCR
0.934273 OCR+DET
0.977831 DET
0.0 OCR
0.928939 OCR+DET
0.978494 DET
0.006246 OCR
0.929882 OCR+DET
0.976098 DET
0.0 OCR
0.927293 OCR+DET
0.97569 DET
0.0 OCR
0.926906 OCR+DET
0.974397 DET
0.0 OCR
0.925677 OCR+DET
0.977808 DET
0.710467 OCR
0.964441 OCR+DET
0.982937 DET
0.0 OCR
0.93379 OCR+DET
0.976836 DET
0.0 OCR
0.927995 OCR+DET
0.975924 DET
0.0 OCR
0.927128 OCR+DET


[OCR+DET to CSV rows]: 100%|██████████| 843/843 [01:21<00:00, 10.36it/s]

0.969269 DET
0.180633 OCR
0.929837 OCR+DET
0.973407 DET
0.0 OCR
0.924737 OCR+DET
0.972152 DET
0.0 OCR
0.923544 OCR+DET
0.970576 DET
0.0 OCR
0.922047 OCR+DET
0.971909 DET
0.796007 OCR
0.963114 OCR+DET
0.96572 DET
0.151672 OCR
0.925017 OCR+DET
0.97206 DET
0.0 OCR
0.923457 OCR+DET
0.968706 DET
0.0 OCR
0.920271 OCR+DET
0.970047 DET
0.170547 OCR
0.930072 OCR+DET
0.973932 DET
0.0 OCR
0.925235 OCR+DET
0.97027 DET
0.0 OCR
0.921757 OCR+DET
0.964932 DET
0.0 OCR
0.916685 OCR+DET
총 rows: 3233


In [28]:
# =========================================================
# 셀 3. CSV 저장
# =========================================================
import pandas as pd
import os

FINAL_SUB_CSV = os.path.join("./work_training", "submission_final.csv")

sub_df = pd.DataFrame(rows_for_csv, columns=[
    "annotation_id","image_id","category_id",
    "bbox_x","bbox_y","bbox_w","bbox_h","score"
])

sub_df.to_csv(FINAL_SUB_CSV, index=False, encoding="utf-8-sig")
print("✅ 최종 제출 CSV 저장 완료 ->", FINAL_SUB_CSV)
print(sub_df.head())

✅ 최종 제출 CSV 저장 완료 -> ./work_training/submission_final.csv
   annotation_id image_id  category_id  bbox_x  bbox_y  bbox_w  bbox_h  \
0              1        1         1899  158.34  247.55  200.95  132.34   
1              2        1        16550  554.39   70.64  395.93  414.31   
2              3        1        27925  598.51  666.72  258.29  495.36   
3              4        1        24849  170.41  735.68  185.23  301.70   
4              5       10        16547  103.63  798.77  240.02  254.54   

      score  
0  0.937168  
1  0.936392  
2  0.927875  
3  0.918414  
4  0.936126  


In [17]:
import glob, os

cands = glob.glob(os.path.join(WORK_DIR, "**", "weights", "best.pt"), recursive=True)
cands_last = glob.glob(os.path.join(WORK_DIR, "**", "weights", "last.pt"), recursive=True)

print("=== BEST CANDIDATES ===")
for c in cands:
    print(c)

print("\n=== LAST CANDIDATES ===")
for c in cands_last:
    print(c)

=== BEST CANDIDATES ===
./work_training/fold1_yolov8s2/weights/best.pt
./work_training/fold2_yolov8s2/weights/best.pt
./work_training/fold2_yolov8m2/weights/best.pt
./work_training/fold1_yolov8m2/weights/best.pt
./work_training/fold0_yolov8m2/weights/best.pt
./work_training/fold0_yolov8s2/weights/best.pt

=== LAST CANDIDATES ===
./work_training/fold1_yolov8s2/weights/last.pt
./work_training/fold2_yolov8s2/weights/last.pt
./work_training/fold2_yolov8m2/weights/last.pt
./work_training/fold1_yolov8m2/weights/last.pt
./work_training/fold0_yolov8m2/weights/last.pt
./work_training/fold0_yolov8s2/weights/last.pt


In [13]:
import os, glob
from pathlib import Path

def rebuild_train_artifacts_from_fs(work_dir, model_list, folds):
    # work_training 아래 run 디렉토리들 스캔해서 foldX_yolov8s* 형태인 것들만 모으는 방식
    rebuilt = []

    # 예: ./work_training/fold0_yolov8s2/weights/best.pt
    # 폴더 이름 패턴은 fold{fold_idx}_{model_stem}*
    all_run_dirs = [d for d in glob.glob(os.path.join(work_dir, "*")) if os.path.isdir(d)]

    for run_dir in all_run_dirs:
        run_name = os.path.basename(run_dir)

        # run_name 예: fold0_yolov8s2
        # 이걸 fold 번호와 모델 스템으로 파싱
        # 1) fold번호 찾기
        if not run_name.startswith("fold"):
            continue
        try:
            # fold0_yolov8s2 -> "0" 추출
            after_fold = run_name[len("fold"):]             # '0_yolov8s2'
            fold_str = after_fold.split("_",1)[0]           # '0'
            fold_idx = int(fold_str)
        except:
            continue

        # 2) 모델 스템 추정
        # fold0_yolov8s2 -> '_yolov8s2'
        after_foldnum = after_fold.split("_",1)[1]          # 'yolov8s2'
        # 모델 후보들: yolov8s, yolov8m, yolov8l ...
        model_match = None
        for m in model_list:
            stem = Path(m).stem  # 'yolov8s'
            if after_foldnum.startswith(stem):
                model_match = m
                break
        if model_match is None:
            continue

        best_pt = os.path.join(run_dir, "weights", "best.pt")
        last_pt = os.path.join(run_dir, "weights", "last.pt")
        hist_json = os.path.join(run_dir, f"{run_name}_history.json")
        if not os.path.isfile(hist_json):
            # history_json 이름이 run_name_history.json로 저장 안 된 경우
            # 이전 셀에서 hist_path를 run_dir 기반으로 썼으니까
            # 혹시 fold0_yolov8s_history.json 이런 이름일 수도 있어서 fallback
            alt_hist = glob.glob(os.path.join(run_dir, "*_history.json"))
            hist_json = alt_hist[0] if alt_hist else None

        rebuilt.append({
            "fold": fold_idx,
            "model": model_match,
            "run_name": run_name,
            "run_dir": run_dir,
            "best_pt": best_pt if os.path.isfile(best_pt) else None,
            "last_pt": last_pt if os.path.isfile(last_pt) else None,
            "history_json": hist_json if hist_json and os.path.isfile(hist_json) else None
        })

    # 최신 run 우선 정렬: run_dir mtime 기준
    rebuilt.sort(key=lambda x: os.path.getmtime(x["run_dir"]), reverse=True)
    return rebuilt

# 실행
rebuilt_artifacts = rebuild_train_artifacts_from_fs(
    WORK_DIR,
    MODEL_LIST,
    folds=range(len(FOLD_DIRS))
)

print("🔥 rebuilt_artifacts:")
for r in rebuilt_artifacts:
    print(r)

🔥 rebuilt_artifacts:
{'fold': 2, 'model': 'yolov8m.pt', 'run_name': 'fold2_yolov8m', 'run_dir': './work_training/fold2_yolov8m', 'best_pt': None, 'last_pt': None, 'history_json': './work_training/fold2_yolov8m/fold2_yolov8m_history.json'}
{'fold': 2, 'model': 'yolov8m.pt', 'run_name': 'fold2_yolov8m2', 'run_dir': './work_training/fold2_yolov8m2', 'best_pt': './work_training/fold2_yolov8m2/weights/best.pt', 'last_pt': './work_training/fold2_yolov8m2/weights/last.pt', 'history_json': None}
{'fold': 2, 'model': 'yolov8s.pt', 'run_name': 'fold2_yolov8s', 'run_dir': './work_training/fold2_yolov8s', 'best_pt': None, 'last_pt': None, 'history_json': './work_training/fold2_yolov8s/fold2_yolov8s_history.json'}
{'fold': 2, 'model': 'yolov8s.pt', 'run_name': 'fold2_yolov8s2', 'run_dir': './work_training/fold2_yolov8s2', 'best_pt': './work_training/fold2_yolov8s2/weights/best.pt', 'last_pt': './work_training/fold2_yolov8s2/weights/last.pt', 'history_json': None}
{'fold': 1, 'model': 'yolov8m.pt', 

In [14]:
ensemble_members = []

for art in rebuilt_artifacts:
    # best 우선, 없으면 last로라도
    cand_weight = None
    if art["best_pt"] is not None:
        cand_weight = art["best_pt"]
    elif art["last_pt"] is not None:
        cand_weight = art["last_pt"]

    if cand_weight is not None and os.path.isfile(cand_weight):
        ensemble_members.append({
            "fold": art["fold"],
            "model": art["model"],
            "weight": cand_weight
        })

print("앙상블 멤버 수:", len(ensemble_members))
for m in ensemble_members:
    print(m)

앙상블 멤버 수: 6
{'fold': 2, 'model': 'yolov8m.pt', 'weight': './work_training/fold2_yolov8m2/weights/best.pt'}
{'fold': 2, 'model': 'yolov8s.pt', 'weight': './work_training/fold2_yolov8s2/weights/best.pt'}
{'fold': 1, 'model': 'yolov8m.pt', 'weight': './work_training/fold1_yolov8m2/weights/best.pt'}
{'fold': 1, 'model': 'yolov8s.pt', 'weight': './work_training/fold1_yolov8s2/weights/best.pt'}
{'fold': 0, 'model': 'yolov8m.pt', 'weight': './work_training/fold0_yolov8m2/weights/best.pt'}
{'fold': 0, 'model': 'yolov8s.pt', 'weight': './work_training/fold0_yolov8s2/weights/best.pt'}


In [15]:
def infer_yolo_weight_over_test(test_dir, weight_path, img_size, idx2cid):
    """
    특정 YOLO weight 하나로 test_dir 전부 추론해서
    {img_path: {"boxes":..., "scores":..., "cat_ids":...}} 형태로 리턴
    """
    model = YOLO(weight_path)
    out_map = {}
    test_list = sorted([
        f for f in os.listdir(test_dir)
        if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))
    ])
    for fn in tqdm(test_list, desc=f"[YOLO] {os.path.basename(weight_path)}"):
        ip = os.path.join(test_dir, fn)
        pred = model.predict(
            source=ip,
            imgsz=img_size,
            conf=0.001,
            iou=0.7,
            agnostic_nms=True,
            verbose=False
        )[0]

        if pred.boxes is None or len(pred.boxes)==0:
            out_map[ip] = {
                "boxes": np.zeros((0,4),dtype=np.float32),
                "scores": np.zeros((0,),dtype=np.float32),
                "cat_ids": np.zeros((0,),dtype=np.int32),
            }
            continue

        boxes_xyxy = pred.boxes.xyxy.cpu().numpy().astype(np.float32)
        scores     = pred.boxes.conf.cpu().numpy().astype(np.float32)
        classes    = pred.boxes.cls.cpu().numpy().astype(int)

        cat_ids = np.array([idx2cid[c] for c in classes], dtype=np.int32)

        out_map[ip] = {
            "boxes": boxes_xyxy,
            "scores": scores,
            "cat_ids": cat_ids
        }
    return out_map

all_detector_outputs = []  # 각 엔트리: {"weight":..., "fold":..., "model":..., "map":{...}}

for m in ensemble_members:
    out_map = infer_yolo_weight_over_test(TEST_DIR, m["weight"], IMGSZ, idx2cid)
    all_detector_outputs.append({
        "fold": m["fold"],
        "model": m["model"],
        "weight": m["weight"],
        "map": out_map
    })

print("수집된 detector 출력 수:", len(all_detector_outputs))

[YOLO] best.pt: 100%|██████████| 843/843 [00:40<00:00, 20.75it/s]

수집된 detector 출력 수: 6


In [16]:
# =========================================================
# 셀 B. fold별 results.csv 수집 → 하나의 summary CSV로 저장
# =========================================================
import glob, os
import pandas as pd

def collect_all_results_csv(work_dir):
    """
    work_training 안을 뒤지면서 */results.csv 찾고,
    run_name(=폴더명) / fold / model 추정해서 attach
    """
    rows = []
    run_dirs = [d for d in glob.glob(os.path.join(work_dir, "*")) if os.path.isdir(d)]
    for rd in run_dirs:
        run_name = os.path.basename(rd)
        csv_path = os.path.join(rd, "results.csv")
        if not os.path.isfile(csv_path):
            continue

        # fold / model 추론
        # 예: run_name = "fold2_yolov8m3"
        # -> fold=2, model="yolov8m"
        fold_idx = None
        model_stem = None
        if run_name.startswith("fold"):
            after_fold = run_name[len("fold"):]        # e.g. "2_yolov8m3"
            parts = after_fold.split("_", 1)
            if len(parts) == 2:
                fold_str, model_plus = parts[0], parts[1]  # "2", "yolov8m3"
                try:
                    fold_idx = int(fold_str)
                except:
                    fold_idx = None
                # model 이름 파싱: yolov8s, yolov8m 등 모델 prefix가 MODEL_LIST stem 중 어디까지인지 찾자
                for m in MODEL_LIST:
                    stem = Path(m).stem  # "yolov8m"
                    if model_plus.startswith(stem):
                        model_stem = stem
                        break

        # CSV 로드
        df_local = pd.read_csv(csv_path)
        df_local["run_name"] = run_name
        df_local["fold"] = fold_idx
        df_local["model_stem"] = model_stem

        rows.append(df_local)

    if len(rows) == 0:
        print("결과 CSV를 찾지 못했다.")
        return None

    df_all = pd.concat(rows, ignore_index=True)

    # 폴드별 마지막 epoch만 보고 싶은 경우:
    # groupby(run_name) 후 epoch 최대만 남겨서 성능 비교 테이블 만들 수 있다.
    summary_last = (
        df_all.sort_values(["run_name","epoch"])
             .groupby("run_name")
             .tail(1)
             .reset_index(drop=True)
    )

    out_full = os.path.join(work_dir, "all_results_full.csv")
    out_last = os.path.join(work_dir, "all_results_last_epoch.csv")

    df_all.to_csv(out_full, index=False, encoding="utf-8-sig")
    summary_last.to_csv(out_last, index=False, encoding="utf-8-sig")

    print("📊 Saved:")
    print(" - full history :", out_full)
    print(" - last epochs  :", out_last)

    return df_all, summary_last

df_all, summary_last = collect_all_results_csv(WORK_DIR)

📊 Saved:
 - full history : ./work_training/all_results_full.csv
 - last epochs  : ./work_training/all_results_last_epoch.csv


In [19]:
# =========================================================
# 셀 A. 앙상블(+WBF) → OCR 재랭크 → 최종 submission.csv 생성
# =========================================================
import os, numpy as np, pandas as pd
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import easyocr

########################################
# 0. 하이퍼파라미터
########################################
IOU_THR_WBF = 0.55   # WBF에서 같은 객체로 취급할 IoU 기준
N_TOP = 4            # 이미지당 최종 제출 박스 개수
OCR_W = 0.25         # OCR boost 계수 (score_final = score * (1 + OCR_W * s_ocr))

# OCR 키워드와 스코어 함수 (이전과 동일 컨셉)
KEYWORDS = [
    "mg","tab","tablet","capsule","cap","sr","er","od","cr",
    "정","캡슐","서방정","정제","경질","연질"
]
def ocr_score_from_text(text: str) -> float:
    """
    OCR 텍스트로부터 [0,1] 스코어.
    - 숫자 있으면 가산
    - 키워드 있으면 가산
    - 텍스트 길이 조금 길면 보너스
    """
    if not text:
        return 0.0
    t = text.lower()
    has_digit = any(ch.isdigit() for ch in t)
    has_kw    = any(k in t for k in KEYWORDS)
    length_bonus = min(len(t) / 10.0, 1.0)  # 10자 이상이면 최대 1.0
    raw = (0.5 if has_digit else 0.0) + (0.4 if has_kw else 0.0) + (0.2 * length_bonus)
    raw = float(max(0.0, min(1.0, raw)))
    return raw

def run_easyocr_on_crops(img_path, boxes_xyxy):
    """
    img_path: 원본 이미지 경로
    boxes_xyxy: (N,4) ndarray, [x1,y1,x2,y2]
    return: np.array([ocr_score_0 ... ocr_score_N])
    """
    if len(boxes_xyxy) == 0:
        return np.zeros((0,), dtype=np.float32)

    # easyocr 준비 (ko+en)
    # GPU 있으면 gpu=True, 없으면 False로 자동 fallback
    reader = easyocr.Reader(['ko','en'], gpu=torch.cuda.is_available())

    im = Image.open(img_path).convert("RGB")
    scores = []
    for (x1,y1,x2,y2) in boxes_xyxy:
        # 안전하게 int 클램프
        x1i = max(0, int(x1))
        y1i = max(0, int(y1))
        x2i = max(x1i+1, int(x2))
        y2i = max(y1i+1, int(y2))

        crop = im.crop((x1i,y1i,x2i,y2i))
        try:
            res = reader.readtext(np.array(crop))
            txt = " ".join([r[1] for r in res])
        except Exception:
            txt = ""
        s = ocr_score_from_text(txt)
        scores.append(s)
    return np.array(scores, dtype=np.float32)


########################################
# 1. 앙상블 helper들 (WBF)
########################################
def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    inter_x1 = max(ax1, bx1)
    inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2)
    inter_y2 = min(ay2, by2)
    iw = max(0.0, inter_x2 - inter_x1)
    ih = max(0.0, inter_y2 - inter_y1)
    inter = iw * ih
    area_a = (ax2-ax1)*(ay2-ay1)
    area_b = (bx2-bx1)*(by2-by1)
    union = area_a + area_b - inter
    if union <= 0:
        return 0.0
    return inter/union

def wbf_merge(boxes, scores, cat_ids, iou_thr=0.55):
    """
    boxes: (M,4) xyxy
    scores: (M,)
    cat_ids: (M,)
    IoU >= iou_thr and same cat_id면 같은 그룹으로 묶어서
    score-weighted 평균 box, 평균 score 리턴
    """
    if len(boxes) == 0:
        return (
            np.zeros((0,4), dtype=np.float32),
            np.zeros((0,), dtype=np.float32),
            np.zeros((0,), dtype=np.int32),
        )

    used = [False]*len(boxes)
    mb, ms, mc = [], [], []

    for i in range(len(boxes)):
        if used[i]:
            continue
        group_idx = [i]
        used[i] = True
        for j in range(i+1, len(boxes)):
            if used[j]:
                continue
            if cat_ids[j] != cat_ids[i]:
                continue
            if iou_xyxy(boxes[i], boxes[j]) >= iou_thr:
                group_idx.append(j)
                used[j] = True

        if len(group_idx) == 1:
            k = group_idx[0]
            mb.append(boxes[k])
            ms.append(scores[k])
            mc.append(cat_ids[k])
        else:
            ws = np.array([scores[g] for g in group_idx])
            ws_sum = ws.sum() + 1e-9
            box_stack = np.stack([boxes[g] for g in group_idx], axis=0)
            avg_box = np.sum(box_stack * ws[:,None], axis=0) / ws_sum
            avg_score = float(ws.mean())
            mb.append(avg_box)
            ms.append(avg_score)
            mc.append(cat_ids[i])

    return (
        np.array(mb, dtype=np.float32),
        np.array(ms, dtype=np.float32),
        np.array(mc, dtype=np.int32)
    )

def fuse_detectors_for_all_images(test_dir, all_detector_outputs, iou_thr=0.55):
    """
    all_detector_outputs: 리스트
      [
        { "fold":int, "model":str, "weight":str, "map": { img_path: {boxes,scores,cat_ids} } },
        ...
      ]
    return fused_map:
      { img_path: { "boxes":..., "scores":..., "cat_ids":... } }
    """
    fused = {}
    test_list = sorted([
        f for f in os.listdir(test_dir)
        if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))
    ])
    for fn in tqdm(test_list, desc="[WBF] fuse-all-detectors"):
        ip = os.path.join(test_dir, fn)

        # 모든 detector 결과 concat
        all_boxes  = []
        all_scores = []
        all_cats   = []
        for det in all_detector_outputs:
            det_map = det["map"]
            if ip not in det_map:
                continue
            y = det_map[ip]
            if len(y["boxes"]) == 0:
                continue
            all_boxes.append(y["boxes"])
            all_scores.append(y["scores"])
            all_cats.append(y["cat_ids"])

        if len(all_boxes)==0:
            fused[ip] = {
                "boxes": np.zeros((0,4),dtype=np.float32),
                "scores": np.zeros((0,),dtype=np.float32),
                "cat_ids": np.zeros((0,),dtype=np.int32),
            }
            continue

        all_boxes  = np.concatenate(all_boxes,  axis=0)
        all_scores = np.concatenate(all_scores, axis=0)
        all_cats   = np.concatenate(all_cats,   axis=0)

        mb, ms, mc = wbf_merge(all_boxes, all_scores, all_cats, iou_thr=iou_thr)

        fused[ip] = {
            "boxes": mb,
            "scores": ms,
            "cat_ids": mc
        }

    return fused


########################################
# 2. 앙상블 수행
########################################
# 여기서 all_detector_outputs, TEST_DIR, IMGSZ, idx2cid 가 이미 있다고 가정
# all_detector_outputs는 너가 방금 만든 그 리스트
fused_map = fuse_detectors_for_all_images(TEST_DIR, all_detector_outputs, iou_thr=IOU_THR_WBF)

########################################
# 3. OCR 재랭크 + N_TOP 상위만 뽑아서 CSV 생성
########################################
rows = []
annotation_id = 1

test_list = sorted([
    f for f in os.listdir(TEST_DIR)
    if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))
])

for fn in tqdm(test_list, desc="[OCR+TOPN] build submission"):
    ip = os.path.join(TEST_DIR, fn)
    base_name = Path(fn).stem

    data = fused_map.get(ip, {
        "boxes": np.zeros((0,4),dtype=np.float32),
        "scores": np.zeros((0,),dtype=np.float32),
        "cat_ids": np.zeros((0,),dtype=np.int32),
    })
    boxes_xyxy = data["boxes"]
    scores_det = data["scores"]
    cats       = data["cat_ids"]

    if boxes_xyxy.shape[0] == 0:
        continue

    # OCR 점수 계산
    ocr_scores = run_easyocr_on_crops(ip, boxes_xyxy)

    # 재랭크
    fused_scores = scores_det * (1.0 + OCR_W * ocr_scores)

    # 상위 N_TOP
    order = np.argsort(-fused_scores)[:N_TOP]

    for k in order:
        x1,y1,x2,y2 = boxes_xyxy[k]
        w = x2 - x1
        h = y2 - y1

        print(round(float(scores_det[k]),6), 'DET')
        print(round(float(ocr_scores[k]),6), 'OCR')
        print(round(float(fused_scores[k]),6), 'OCR+DET')
        rows.append({
            "annotation_id": annotation_id,
            "image_id": base_name,
            "category_id": int(cats[k]),  # 이미 cat_ids는 original category_id
            "bbox_x": round(float(x1),2),
            "bbox_y": round(float(y1),2),
            "bbox_w": round(float(w),2),
            "bbox_h": round(float(h),2),
            "score": round(float(fused_scores[k]),6),  # 실제 랭킹에 쓴 점수
            # "score_det": round(float(scores_det[k]),6),      # 디버그용
            # "score_ocr": round(float(ocr_scores[k]),6),      # 디버그용
            # "score_final": round(float(fused_scores[k]),6),  # 실제 랭킹에 쓴 점수
        })
        annotation_id += 1

sub_df = pd.DataFrame(rows, columns=[
    "annotation_id","image_id","category_id",
    "bbox_x","bbox_y","bbox_w","bbox_h",
    "score"
])

out_csv = os.path.join(WORK_DIR, "submission_ensemble_ocr.csv")
sub_df.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("🎯 submission with ensemble+OCR saved:", out_csv)
print("rows =", len(sub_df))

[OCR+TOPN] build submission:   0%|          | 1/843 [00:02<35:52,  2.56s/it]

0.972295 DET
0.6 OCR
1.118139 OCR+DET
0.970684 DET
0.02 OCR
0.975538 OCR+DET
0.829904 DET
0.02 OCR
0.834054 OCR+DET
0.833417 DET
0.0 OCR
0.833417 OCR+DET


[OCR+TOPN] build submission:   0%|          | 2/843 [00:05<35:25,  2.53s/it]

0.966744 DET
0.02 OCR
0.971578 OCR+DET
0.96246 DET
0.0 OCR
0.96246 OCR+DET
0.961749 DET
0.0 OCR
0.961749 OCR+DET
0.812805 DET
0.0 OCR
0.812805 OCR+DET


[OCR+TOPN] build submission:   0%|          | 3/843 [00:07<35:10,  2.51s/it]

...

0.969547 DET
0.0 OCR
0.969547 OCR+DET
0.964452 DET
0.02 OCR
0.969274 OCR+DET
0.962963 DET
0.0 OCR
0.962963 OCR+DET
0.832035 DET
0.0 OCR
0.832035 OCR+DET


[OCR+TOPN] build submission: 100%|█████████▉| 842/843 [35:18<00:02,  2.50s/it]

0.975486 DET
0.02 OCR
0.980364 OCR+DET
0.966215 DET
0.0 OCR
0.966215 OCR+DET
0.959544 DET
0.0 OCR
0.959544 OCR+DET
0.529106 DET
0.0 OCR
0.529106 OCR+DET


[OCR+TOPN] build submission: 100%|██████████| 843/843 [35:20<00:00,  2.52s/it]

0.969634 DET
0.02 OCR
0.974482 OCR+DET
0.962769 DET
0.0 OCR
0.962769 OCR+DET
0.960317 DET
0.0 OCR
0.960317 OCR+DET
0.812002 DET
0.0 OCR
0.812002 OCR+DET
🎯 submission with ensemble+OCR saved: ./work_training/submission_ensemble_ocr.csv
rows = 3341


In [20]:
# =========================================================
# 모든 fold_* 폴더의 results.csv를 모아서 ZIP으로 다운로드
# =========================================================
import os
import glob
from google.colab import files
import zipfile

WORK_DIR = "./work_training"  # 너가 사용하던 경로 유지

# 1️⃣ results.csv 경로 전부 수집
result_csvs = glob.glob(os.path.join(WORK_DIR, "**", "results.csv"), recursive=True)

if len(result_csvs) == 0:
    print("❌ results.csv 파일을 찾지 못했습니다.")
else:
    print(f"✅ 총 {len(result_csvs)}개 results.csv 발견")
    for c in result_csvs:
        print(" -", c)

    # 2️⃣ ZIP 생성
    zip_path = os.path.join(WORK_DIR, "all_results_csv.zip")
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        for fp in result_csvs:
            arcname = os.path.relpath(fp, WORK_DIR)
            zf.write(fp, arcname)
    print(f"📦 압축 완료: {zip_path}")

    # 3️⃣ 다운로드
    files.download(zip_path)

✅ 총 6개 results.csv 발견
 - ./work_training/fold1_yolov8s2/results.csv
 - ./work_training/fold2_yolov8s2/results.csv
 - ./work_training/fold2_yolov8m2/results.csv
 - ./work_training/fold1_yolov8m2/results.csv
 - ./work_training/fold0_yolov8m2/results.csv
 - ./work_training/fold0_yolov8s2/results.csv
📦 압축 완료: ./work_training/all_results_csv.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>